In [2]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg = VGG19(include_top=False, weights='imagenet')
for layer in vgg.layers:
    print(f'{layer.name} --> {layer.output_shape}')

input_1 --> [(None, None, None, 3)]
block1_conv1 --> (None, None, None, 64)
block1_conv2 --> (None, None, None, 64)
block1_pool --> (None, None, None, 64)
block2_conv1 --> (None, None, None, 128)
block2_conv2 --> (None, None, None, 128)
block2_pool --> (None, None, None, 128)
block3_conv1 --> (None, None, None, 256)
block3_conv2 --> (None, None, None, 256)
block3_conv3 --> (None, None, None, 256)
block3_conv4 --> (None, None, None, 256)
block3_pool --> (None, None, None, 256)
block4_conv1 --> (None, None, None, 512)
block4_conv2 --> (None, None, None, 512)
block4_conv3 --> (None, None, None, 512)
block4_conv4 --> (None, None, None, 512)
block4_pool --> (None, None, None, 512)
block5_conv1 --> (None, None, None, 512)
block5_conv2 --> (None, None, None, 512)
block5_conv3 --> (None, None, None, 512)
block5_conv4 --> (None, None, None, 512)
block5_pool --> (None, None, None, 512)
